In [1]:
import cv2
import numpy as np

In [2]:
def save_initial():
    '''Used to save initial picture of the speedometer to be later used for different computations. 
    Saves a "reference.jpg" file in the current working directory'''
    try:
        cap = cv2.VideoCapture(0)
        while True:
            _,frame = cap.read()
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_gray_copy = np.copy(frame_gray)
            cv2.putText(frame_gray_copy, "Press 's' to save image", (0,25), 2, 0.9, (255,255,255), 1, cv2.LINE_AA, False)
            cv2.line(frame_gray_copy, (0,30), (frame_gray_copy.shape[1], 30), (255,255,255))
            cv2.imshow('frame', frame_gray_copy)
            k = cv2.waitKey(1) & 0xFF
            if k in [115, 83]:
                cv2.imwrite('reference.jpg', frame_gray)
                print('Image Saved')
                break
            elif k == 27:
                break
    except:
        print('An error has occurred.')
    cap.release()
    cv2.destroyAllWindows()

In [3]:
def compute_sum_thresh(frame, tolerance_percent=5):
    return_value = {'pixel_sum':-1, 'lower_limit':-1, 'upper_limit':-1}
    return_value['pixel_sum'] = np.sum(frame/ frame.size)
    return_value['lower_limit'] = return_value['pixel_sum']*(1-tolerance_percent/100)
    return_value['upper_limit'] = return_value['pixel_sum']*(1+tolerance_percent/100)
    return return_value

In [7]:
def compare(filename='reference.jpg'):
    tolerance_percent = 2
    # Read the image
    reference_img = cv2.imread(filename, 0)
    
    ref_img_values = compute_sum_thresh(reference_img, tolerance_percent)
    cv2.imshow('Reference Image', reference_img)
    
    print("\nDisplaying reference image.\nPress 'Esc' to exit, any other key to continue.")
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        return
    
    cap = cv2.VideoCapture(0)
    while True:
        _, frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_values = compute_sum_thresh(frame_gray, tolerance_percent)
        
        okng_text = ""
        okng_color = (0,0,0)
        if frame_values['pixel_sum'] > ref_img_values['upper_limit'] or frame_values['pixel_sum'] < ref_img_values['lower_limit']:
            okng_text = 'NG'
            okng_color = (0,0,255)
        else:
            okng_text = 'OK'
            okng_color = (0,255,0)
        
        blackpart = np.zeros((30, frame_gray.shape[1]), dtype='uint8')
        text_blackpart1 = "Desired: [" + str(ref_img_values['lower_limit']) + ', '+ str(ref_img_values['upper_limit']) +']'
        text_blackpart2 = "Current: " + str(frame_values['pixel_sum'])
        
        frame_gray = np.vstack((frame_gray, blackpart))
        
        cv2.putText(frame_gray,text_blackpart1+' '*10+text_blackpart2,(10,frame_gray.shape[0]-10),2,0.4,(255,255,255),1, cv2.LINE_AA)
        
        frame_output = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
        cv2.putText(frame_output, okng_text, (10,25), 2, 1.2, okng_color, 1, cv2.LINE_AA)
        
        print(text_blackpart1, '\t', text_blackpart2)
        cv2.imshow('Output', frame_output)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            cap.release()
            break
    cv2.destroyAllWindows()

In [8]:
save_initial()

Image Saved


In [9]:
compare()


Displaying reference image.
Press 'Esc' to exit, any other key to continue.
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 15.024453125000004
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 9.305361328125002
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 9.15393229166667
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 9.045579427083334
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 0.3465397135416666
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 0.8269824218749999
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 0.8294661458333337
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 2.633248697916666
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 7.63294921875
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 10.515273437500001
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 11.468570963541666
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 17.5

Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.93687174479163
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 148.0178385416667
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.73694010416665
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.49346028645834
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.83433593750001
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 148.09124023437496
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.62139973958332
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.9906477864583
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.88701497395834
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.835146484375
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.94364583333333
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.823857421875
Desired: [153.63998170572916, 159.9110013671875] 	 Current

Desired: [153.63998170572916, 159.9110013671875] 	 Current: 148.10895833333333
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 148.36643880208334
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 148.02637695312498
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 148.18696289062498
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.97771809895832
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.89065104166662
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.74345052083336
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 147.73232747395835
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 146.14030924479172
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 145.94149739583332
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 146.2650065104167
Desired: [153.63998170572916, 159.9110013671875] 	 Current: 145.8065006510417
Desired: [153.63998170572916, 159.9110013671875] 	 Cur